In [ ]:
# ==========================
# Marketing Campaign Analysis
# ==========================

# Step 1: Import Libraries
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Load CSV
df = pd.read_csv('../data/marketing_campaign_synthetic.csv')
df.head()  # Preview first 5 rows

# Step 3: Clean Data
df['date'] = pd.to_datetime(df['date'])  # Convert date column
df = df.dropna(subset=['channel', 'campaign_id'])  # Remove missing channels/campaign_id

# Optional: Replace negative values with 0
numeric_cols = ['impressions', 'clicks', 'leads', 'conversions', 'cost', 'revenue']
for col in numeric_cols:
    df[col] = df[col].apply(lambda x: max(x,0))

# Step 4: Add Key Metrics
df['CTR'] = df['clicks'] / df['impressions']
df['conversion_rate'] = df['conversions'] / df['leads'].replace(0, pd.NA)
df['CAC'] = df['cost'] / df['conversions'].replace(0, pd.NA)
df['ROI'] = (df['revenue'] - df['cost']) / df['cost'] * 100

# Step 5: Summarize KPIs by Channel
kpis = df.groupby('channel').agg({
    'impressions':'sum',
    'clicks':'sum',
    'leads':'sum',
    'conversions':'sum',
    'cost':'sum',
    'revenue':'sum',
    'CTR':'mean',
    'conversion_rate':'mean',
    'CAC':'mean',
    'ROI':'mean'
}).reset_index()

print("===== KPI Summary by Channel =====")
kpis

# Step 6: Visualize KPIs

# 6a: ROI by Channel
plt.figure(figsize=(8,5))
plt.bar(kpis['channel'], kpis['ROI'], color='skyblue')
plt.title("ROI % by Channel")
plt.ylabel("ROI %")
plt.xticks(rotation=45)
plt.show()

# 6b: CAC vs Conversion Rate
plt.figure(figsize=(8,5))
plt.scatter(kpis['CAC'], kpis['conversion_rate'], color='orange', s=100)
plt.xlabel("CAC ($)")
plt.ylabel("Conversion Rate")
plt.title("CAC vs Conversion Rate by Channel")
plt.show()

# 6c: CTR by Channel
plt.figure(figsize=(8,5))
plt.bar(kpis['channel'], kpis['CTR'], color='green')
plt.title("Click Through Rate by Channel")
plt.ylabel("CTR")
plt.xticks(rotation=45)
plt.show()

# Step 7: Save KPI Summary
kpis.to_csv('../data/channel_kpis.csv', index=False)
print("KPI summary saved to ../data/channel_kpis.csv")
